In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline

In [2]:
def GetEmbeddings(path='./student_code/supportfiles/GloVe300.d'):
    GloVe = pickle.load(open(path,'rb'))
    W2ID = {w:i for i,w in enumerate(sorted(list(GloVe.keys())))}
    EMB = torch.nn.Embedding(len(W2ID),300)
    EMB.weight.requires_grad=False
    GloVeW = np.vstack([GloVe[w] for w in W2ID])
    EMB.weight.data.copy_(torch.from_numpy(GloVeW))
    return W2ID, EMB
W2ID,EMB = GetEmbeddings('./SupFiles/GloVe300.d')

In [3]:
def filterit(s,W2ID):
    s=s.lower()
    S=''
    for c in s:
        if c in ' abcdefghijklmnopqrstuvwxyz0123456789':
        	S+=c
    S = " ".join([x  if x and x in W2ID else "<unk>" for x in S.split()])
    return S


def Sentence2Embeddings(sentence,W2ID,EMB):
    if type(sentence)==str:
        sentence = filterit(sentence, W2ID)
        #print(sentence)
        IDS = torch.tensor([W2ID[i] for i in sentence.split(" ")])
        return EMB(IDS)
    if type(sentence)==list:
        sembs = []
        for sent in sentence:
            sent = filterit(sent,W2ID)
            IDS = torch.tensor([W2ID[i] for i in sent.split(" ")])
            sembs.append(EMB(IDS))
        sembs = torch.nn.utils.rnn.pad_sequence(sembs,batch_first=True)
        return sembs

In [4]:
class SentimentA(torch.nn.Module):
    def __init__(self,nE=300,nH=512,nL=2):
        super().__init__()
        self.rnn = torch.nn.LSTM(nE,nH,nL,batch_first=True)
        self.fc = torch.nn.Linear(nH,1)
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self,X,*kwargs):
        outs,(h,c) = self.rnn(X,*kwargs)
        out = self.fc(h[-1,:,:])#(outs[:,-1,:])
        out = self.sigmoid(out)
        return out

In [10]:
SA = SentimentA()
SA.load_state_dict(torch.load('./SentimentAnalysisModel',map_location='cpu'))
if torch.cuda.is_available(): SA = SA.cuda()
SA.eval()

SentimentA(
  (rnn): LSTM(300, 512, num_layers=2, batch_first=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [11]:
sent = ['Hello all the movie is really bad', 
        'the movie is not bad at all',
       'the movie is not good at all',
       'the movie is not good but not bad too',
       'I am really saddened by his demise']
emb = Sentence2Embeddings(sent,W2ID,EMB)
if torch.cuda.is_available(): emb=emb.cuda()
print(2*(SA(emb)-0.5))

tensor([[-0.9650],
        [ 0.9996],
        [-0.9832],
        [ 0.8660],
        [-0.9999]], grad_fn=<MulBackward0>)


In [44]:
def Senti(sent):
    emb = Sentence2Embeddings(sent,W2ID,EMB)
    if torch.cuda.is_available(): emb=emb.cuda()
    return 2*(SA(emb)-0.5).detach().numpy()

In [45]:
Senti(sent)

array([[-0.96499544],
       [ 0.99962556],
       [-0.98324347],
       [ 0.86596274],
       [-0.9999136 ]], dtype=float32)